In [2]:
import os.path
import datetime

import numpy  as np
import pandas as pd

In [3]:
# Constants

dataDir    = '../Data'
pickleDir  = '../Data/Pickle'
pickleDir1 = os.path.join(pickleDir,'Result-01')
pickleDir2 = os.path.join(pickleDir,'Result-02')
cols1      = ['user_id','dt0m','is_booking','cnt','hotel_cluster']

nLargest = 5
nCluster = 100
cols1

['user_id', 'dt0m', 'is_booking', 'cnt', 'hotel_cluster']

In [4]:
# Read train.csv or if exists train-user-id-book then picle saved file
train_pickle = os.path.join(pickleDir,'trainx-user-id-dt0m-book.pkl')
if os.path.exists(train_pickle) :
    trainx = pd.read_pickle(train_pickle)
else :
    trainx = pd.read_csv(os.path.join(dataDir,'trainExt.csv'),usecols=cols1,
                     dtype={'user_id':int,'dt0m':int,'is_booking':int,'cnt':int,'hotel_cluster':int})
    print('trainx.shape = ',trainx.shape)
    trainx.index = [trainx.user_id,trainx.dt0m,trainx.is_booking]
    print('trainx.columns = ',trainx.columns)
    trainx = trainx.sort_index(kind='quicksort')
    trainx.to_pickle(train_pickle)
print(trainx.head())
print(trainx.tail())

                         user_id  dt0m  is_booking  cnt  hotel_cluster
user_id dt0m is_booking                                               
0       9    0                 0     9           0    1             19
        12   0                 0    12           0    1             98
             0                 0    12           0    1             40
             0                 0    12           0    1             56
             0                 0    12           0    2             72
                         user_id  dt0m  is_booking  cnt  hotel_cluster
user_id dt0m is_booking                                               
1198784 4    0           1198784     4           0    2             26
        8    0           1198784     8           0    1             79
             1           1198784     8           1    1             79
1198785 7    0           1198785     7           0    2             76
             0           1198785     7           0    1             76


In [5]:
trainx[trainx.user_id==0]

user_id  dt0m  is_booking  cnt  hotel_cluster
user_id dt0m is_booking                                               
0       9    0                 0     9           0    1             19
        12   0                 0    12           0    1             98
             0                 0    12           0    1             40
             0                 0    12           0    1             56
             0                 0    12           0    2             72
             0                 0    12           0    1             28

In [6]:
trainx.head()

user_id  dt0m  is_booking  cnt  hotel_cluster
user_id dt0m is_booking                                               
0       9    0                 0     9           0    1             19
        12   0                 0    12           0    1             98
             0                 0    12           0    1             40
             0                 0    12           0    1             56
             0                 0    12           0    2             72

In [7]:
trainx.tail()

user_id  dt0m  is_booking  cnt  hotel_cluster
user_id dt0m is_booking                                               
1198784 4    0           1198784     4           0    2             26
        8    0           1198784     8           0    1             79
             1           1198784     8           1    1             79
1198785 7    0           1198785     7           0    2             76
             0           1198785     7           0    1             76

In [20]:
# build list for partitional work
lpiece = 19000
lmin = trainx['user_id'].min()
lmax = trainx['user_id'].max()
print(lmin,lmax)

lend = lmax+1
l = [i for i in range(0,lend,lpiece)]
if (l[len(l)-1]!=lend) : l.append(lend)
    
mm = [(l[i],l[i+1]) for i in range(len(l)-1)]
print(len(mm),mm[0:5],mm[-3:])

0 1198785
64 [(0, 19000), (19000, 38000), (38000, 57000), (57000, 76000), (76000, 95000)] [(1159000, 1178000), (1178000, 1197000), (1197000, 1198786)]


In [15]:
# Function for apply function: count counts, and build arrays for hostel_clusters
i        = -1
x00x     = [-1]*nLargest

dcol = ['count','countCnt'] + ['n'+str(i) for i in range(nLargest)]  + ['v'+str(i) for i in range(nLargest)] 

def fApply (xgroup) :
    global i,nLargest,nCluster
    ygroup = xgroup[['hotel_cluster','cnt']].groupby('hotel_cluster').sum().reset_index()
    d00  = np.zeros((nCluster),dtype=np.int16); d00[ygroup['hotel_cluster']]+=ygroup['cnt']
    dsum = xgroup['cnt'].sum()
    dcnt = len(xgroup)
    d05  = pd.Series(d00); 
    n05=d05[d05>0].nlargest(nLargest).index.tolist()+x00x; n05=n05[0:nLargest];
    v05=d05[d05>0].nlargest(nLargest).tolist()+x00x;       v05=v05[0:nLargest];
    ddd  = [dcnt,dsum] + n05 + v05 # + d00.tolist()
    #i   += 1
    #if (i%10000==0) : print('5 :',datetime.datetime.now()," i = ",i," len-xgroup =",len(xgroup),ddd)

    
    return(pd.DataFrame(ddd,index=dcol,dtype=int).T)



In [18]:
# Function for split and join arrays for is_booking {True,False}
def fJoin (transx,nn,filepickle) :
    trans0 = transx.query('is_booking==0')[['user_id','dt0m','is_booking','count','countCnt']+ \
                                              ['n'+str(i) for i in range(nn)]+['v'+str(i) for i in range(nn)]]
    trans0.index = [trans0.user_id, trans0.dt0m]
    trans0.dtype = int
    #print(trans0.head())

    trans1 = transx.query('is_booking==1')[['user_id','dt0m','is_booking','count','countCnt']+ \
                                             ['n'+str(i) for i in range(nn)]+['v'+str(i) for i in range(nn)]]
    trans1.index = [trans1.user_id, trans1.dt0m]
    trans1.dtype = int
    #print(trans1.head())

    trans01 = pd.concat([trans0,trans1],axis=1)
    trans01.columns = ['user_id','dt0m','bo0','count0','countCnt0']+['b0n'+str(i) for i in range(nn)]+['b0v'+str(i) for i in range(nn)]+ \
                        ['h1','dt0m1','bo1','count1','countCnt1']+['b1n'+str(i) for i in range(nn)]+['b1v'+str(i) for i in range(nn)]
    trans01 = trans01[['user_id','dt0m','count0','countCnt0','count1','countCnt1'] + \
                      ['b0n'+str(i) for i in range(nn)]+['b1n'+str(i) for i in range(nn)]+ \
                      ['b0v'+str(i) for i in range(nn)]+['b1v'+str(i) for i in range(nn)]]
    trans01.dtype = int
    
    trans01.to_pickle(filepickle)
    
    return(trans01)


In [ ]:
# Main body for calculate partition and save in file
fres = []
i1,i2 = 10000,20000
for i1,i2 in mm :
    print('4 :',datetime.datetime.now(),' i1,i2=',i1,i2)
    cond = '('+str(i1)+'<=user_id)&(user_id<'+str(i2)+')'
    grouped100 = trainx.query(cond).groupby(['user_id','dt0m','is_booking']); 
    #print('user+book =',len(grouped100))
    #print('before fApply:',datetime.datetime.now())
    i       = -1
    trans   = grouped100.apply(fApply);
    #print('after fApply:',datetime.datetime.now())
    trans   = trans.reset_index()
    #print('before fJoin:',datetime.datetime.now())
    ffile   = os.path.join(pickleDir,'trans01-'+str(i1+1000000000)+'.pck')
    trans01 = fJoin(trans,nLargest,ffile)
    fres.append(ffile)
    print('6 :',datetime.datetime.now(),trans01.shape)
    
print('7 :',datetime.datetime.now())
print('fres=',len(fres))
trans01.head()

4 : 2016-05-28 05:24:47.223841  i1,i2= 0 19000


C:\Anaconda3-64\lib\site-packages\ipykernel\__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


5 : 2016-05-28 05:24:50.219811  i =  0  len-xgroup = 1 [1, 1, 19, -1, -1, -1, -1, 1, -1, -1, -1, -1]
5 : 2016-05-28 05:26:01.811403  i =  10000  len-xgroup = 1 [1, 1, 37, -1, -1, -1, -1, 1, -1, -1, -1, -1]
5 : 2016-05-28 05:27:18.306478  i =  20000  len-xgroup = 11 [11, 13, 59, 3, 40, 5, 30, 3, 2, 2, 1, 1]
5 : 2016-05-28 05:28:44.111636  i =  30000  len-xgroup = 18 [18, 43, 85, 36, 62, 57, 99, 19, 7, 5, 4, 4]


In [113]:
len(trans01)
trans01.tail()

,user_id,count0,countCnt0,count1,countCnt1,b0n0,b0n1,b0n2,b0n3,b0n4,...,b0v0,b0v1,b0v2,b0v3,b0v4,b1v0,b1v1,b1v2,b1v3,b1v4
user_id,,,,,,,,,,,,,,,,,,,,,
29995,29995,3,5,NaN,NaN,39,40,48,-1,-1,...,3,1,1,-1,-1,NaN,NaN,NaN,NaN,NaN
29996,29996,7,9,1.0,1.0,23,50,72,91,-1,...,6,1,1,1,-1,1.0,-1.0,-1.0,-1.0,-1.0
29997,29997,9,14,1.0,1.0,42,48,91,53,94,...,4,4,3,2,1,1.0,-1.0,-1.0,-1.0,-1.0
29998,29998,14,18,NaN,NaN,70,21,4,10,17,...,5,3,1,1,1,NaN,NaN,NaN,NaN,NaN
29999,29999,3,3,2.0,2.0,39,42,91,-1,-1,...,1,1,1,-1,-1,1.0,1.0,-1.0,-1.0,-1.0


In [120]:
trainx.query('user_id==1190000')['cnt'].count()

60

In [134]:
import os
xres = [os.path.join(pickleDir,iFile) for iFile in os.listdir(pickleDir) if (iFile[0:7]=='trans01')]
fres=xres
len(fres)

120

In [135]:
# Read files and build one dataframe
ff  = pd.read_pickle(fres[0])
for ifile in fres[1:len(fres)] :
    ff1 = pd.read_pickle(ifile)
    ff  = pd.concat([ff,ff1])
    del ff1
print(ff.shape)
ff.tail()

(1198786, 25)


,user_id,count0,countCnt0,count1,countCnt1,b0n0,b0n1,b0n2,b0n3,b0n4,...,b0v0,b0v1,b0v2,b0v3,b0v4,b1v0,b1v1,b1v2,b1v3,b1v4
user_id,,,,,,,,,,,,,,,,,,,,,
1198781,1198781,10,18,NaN,NaN,70,83,25,39,40,...,6,3,2,2,2,NaN,NaN,NaN,NaN,NaN
1198782,1198782,3,7,NaN,NaN,77,37,55,-1,-1,...,3,2,2,-1,-1,NaN,NaN,NaN,NaN,NaN
1198783,1198783,23,33,4.0,4.0,91,48,39,40,94,...,7,6,3,3,3,1.0,1.0,1.0,1.0,-1.0
1198784,1198784,4,10,1.0,1.0,68,26,79,95,-1,...,6,2,1,1,-1,1.0,-1.0,-1.0,-1.0,-1.0
1198785,1198785,2,3,NaN,NaN,76,-1,-1,-1,-1,...,3,-1,-1,-1,-1,NaN,NaN,NaN,NaN,NaN


In [136]:
ff.shape

(1198786, 25)

In [137]:
# Save result in file
print(ff.shape)
ff.to_pickle(os.path.join(pickleDir,'trans01-user-id-all.pck'))
#del ff

(1198786, 25)


In [138]:
del ff
del trainx